### 1 - Import libraries

In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, random_split
import copy

# For reproducibility
np.random.seed(42)
torch.manual_seed(42)

### 2 - Prepare Your Data

In [2]:
# Assuming your data preparation steps are done here
num_simulations = 100
train_ratio = 0.8

# Load npy data from training_data folder
pressure_data = np.load('training_data/pressure_data.npy')
velocity_data = np.load('training_data/velocity_data.npy')

pressure_flattened = pressure_data.reshape(num_simulations, -1)
velocity_flattened = velocity_data.reshape(num_simulations, -1)

data_flattened = np.hstack((pressure_flattened, velocity_flattened))

# Convert to PyTorch tensors
data_tensor = torch.tensor(data_flattened, dtype=torch.float32)

# Split the data
train_size = int(train_ratio * num_simulations)
test_size = num_simulations - train_size
train_dataset, test_dataset = random_split(data_tensor, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

### 3 - Define the Autoencoder Model

In [3]:
class Autoencoder(nn.Module):
    def __init__(self, input_dim, latent_dim):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 1024),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, latent_dim),
            nn.ReLU()
        )
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, 256),
            nn.ReLU(),
            nn.Linear(256, 512),
            nn.ReLU(),
            nn.Linear(512, 1024),
            nn.ReLU(),
            nn.Linear(1024, input_dim),
            nn.Sigmoid()
        )

    def forward(self, x):
        latent = self.encoder(x)
        reconstructed = self.decoder(latent)
        return reconstructed

# Define model, loss, and optimizer
input_dim = data_flattened.shape[1]  # 15000
latent_dim = 256
autoencoder = Autoencoder(input_dim, latent_dim)
criterion = nn.MSELoss()
optimizer = optim.Adam(autoencoder.parameters(), lr=0.001)


### 4 - Train the Autoencoder

In [4]:
# Training the Autoencoder
model_save_path = 'model_saved'

num_epochs = 20
best_model_wts = copy.deepcopy(autoencoder.state_dict())
best_loss = float('inf')

for epoch in range(num_epochs):
    autoencoder.train()
    train_loss = 0.0

    for data in train_loader:
        inputs = data
        optimizer.zero_grad()
        outputs = autoencoder(inputs)
        loss = criterion(outputs, inputs)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * inputs.size(0)

    train_loss /= len(train_loader.dataset)

    autoencoder.eval()
    val_loss = 0.0

    with torch.no_grad():
        for data in test_loader:
            inputs = data
            outputs = autoencoder(inputs)
            loss = criterion(outputs, inputs)
            val_loss += loss.item() * inputs.size(0)

    val_loss /= len(test_loader.dataset)

    print(f'Epoch {epoch + 1}/{num_epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}')

    # Save the best model
    if val_loss < best_loss:
        best_loss = val_loss
        best_model_wts = copy.deepcopy(autoencoder.state_dict())

# Load the best model weights
autoencoder.load_state_dict(best_model_wts)

# Save the trained autoencoder
torch.save(autoencoder.state_dict(), f'{model_save_path}/autoencoder_10epoches.pth')

# Extract the encoder
encoder = autoencoder.encoder

# Extract and store the decoder weights
decoder_weights = []
for i in range(num_simulations):
    autoencoder.decoder.load_state_dict(autoencoder.decoder.state_dict())
    decoder_weights.append(copy.deepcopy(autoencoder.decoder.state_dict()))

# Save the decoder weights for future use
torch.save(decoder_weights, f'{model_save_path}/decoder_weights.pth')

Epoch 1/20, Train Loss: 0.2053, Val Loss: 0.0778
Epoch 2/20, Train Loss: 0.0549, Val Loss: 0.0430
Epoch 3/20, Train Loss: 0.0480, Val Loss: 0.0563
Epoch 4/20, Train Loss: 0.0570, Val Loss: 0.0578
Epoch 5/20, Train Loss: 0.0574, Val Loss: 0.0561
Epoch 6/20, Train Loss: 0.0554, Val Loss: 0.0531
Epoch 7/20, Train Loss: 0.0522, Val Loss: 0.0491
Epoch 8/20, Train Loss: 0.0462, Val Loss: 0.0281
Epoch 9/20, Train Loss: 0.0379, Val Loss: 0.0258
Epoch 10/20, Train Loss: 0.0277, Val Loss: 0.0267
Epoch 11/20, Train Loss: 0.0239, Val Loss: 0.0198
Epoch 12/20, Train Loss: 0.0189, Val Loss: 0.0167
Epoch 13/20, Train Loss: 0.0163, Val Loss: 0.0164
Epoch 14/20, Train Loss: 0.0163, Val Loss: 0.0162
Epoch 15/20, Train Loss: 0.0160, Val Loss: 0.0157
Epoch 16/20, Train Loss: 0.0155, Val Loss: 0.0152
Epoch 17/20, Train Loss: 0.0151, Val Loss: 0.0150
Epoch 18/20, Train Loss: 0.0148, Val Loss: 0.0147
Epoch 19/20, Train Loss: 0.0147, Val Loss: 0.0146
Epoch 20/20, Train Loss: 0.0145, Val Loss: 0.0145


### 5. Train RBF Interpolator on Decoder Weights

In [5]:
from scipy.interpolate import Rbf

In [7]:
# Load design parameters from training_data folder
design_parameters = np.load('training_data/displacement_data.npy')
design_parameters_tensor = torch.tensor(design_parameters, dtype=torch.float32)

# Load the stored decoder weights
decoder_weights = torch.load(f'{model_save_path}/decoder_weights.pth')

# Flatten the decoder weights for RBF interpolation
flattened_weights = []
for weights in decoder_weights:
    flat_weights = []
    for param in weights.values():
        flat_weights.append(param.view(-1).numpy())
    flattened_weights.append(np.concatenate(flat_weights))

flattened_weights = np.array(flattened_weights)

# Train the RBF interpolator
rbf = Rbf(*design_parameters_tensor.T.numpy(), flattened_weights.T, function='linear', mode='N-D')

: 

### 6. Interpolate New Decoder Weights for New Design Parameters

In [ ]:
from pygem import FFD

def generate_new_design_parameters():

    ffd = FFD([5, 5, 2])

    # create the bounding box
    ffd.box_origin = np.array([-0.01, -0.01, -0.001])
    ffd.box_length = np.array([0.02, 0.02, 0.002])

    # define the weight matrix. The boundary control points are fixed
    weights = np.zeros(ffd.array_mu_x.shape)
    weights[1:-1, 1:-1, :] = 1

    # define the number of mesh samples, and the displacement ratio for the control points 
    disp_ratio=0.1

    ffd.array_mu_x=disp_ratio*np.random.uniform(-1, 1, size=ffd.array_mu_x.shape)*weights
    ffd.array_mu_y=disp_ratio*np.random.uniform(-1, 1, size=ffd.array_mu_x.shape)*weights

    ffd.array_mu_x[:, :, 0] = ffd.array_mu_x[:, :, 1]
    ffd.array_mu_y[:, :, 0] = ffd.array_mu_y[:, :, 1]

    displacement_data = np.array([ffd.array_mu_x, ffd.array_mu_y, ffd.array_mu_z]).reshape(1, -1)

    return displacement_data

In [ ]:
# Example new design parameter
new_design_param = generate_new_design_parameters()

# Predict new decoder weights using the RBF interpolator
new_flattened_weights = rbf(*new_design_param)

# Reshape the interpolated weights back to the original shapes
def reshape_weights(flat_weights, template_weights):
    new_weights = {}
    start = 0
    for key, param in template_weights.items():
        param_shape = param.shape
        param_size = param.numel()
        new_weights[key] = torch.tensor(flat_weights[start:start + param_size]).view(param_shape)
        start += param_size
    return new_weights

# Use the first set of decoder weights as a template
template_weights = decoder_weights[0]
new_decoder_weights = reshape_weights(new_flattened_weights, template_weights)

# Load the interpolated weights into a new decoder
new_autoencoder = Autoencoder(input_dim, latent_dim)
new_autoencoder.decoder.load_state_dict(new_decoder_weights)

### 7. Reconstruct Using the New Decoder

In [ ]:
# Obtain the latent representation of the data_tensor
with torch.no_grad():
    latent_representation = autoencoder.encoder(data_tensor)  

# Reconstruct the data using the new decoder
with torch.no_grad():
    reconstructed_data = new_autoencoder.decoder(latent_representation)

# Reshape the reconstructed data if needed
reconstructed_data_reshaped = reconstructed_data.view(-1, 3)  # Example reshape if needed
print(reconstructed_data_reshaped)
